In this notebook I will test the model proposed in the ensemble learning lecture.
Chapter with explenation about the algorithm behind the lightGBM, will write this after the coding.

LightGBM is good in our case because it can deal with categorical values. We will start with importing the data, the data is already divided into test and training data.

In [200]:
import lightgbm as lgb
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
plt.style.use('ggplot')

I want to use import_data function from another directory, and make the data ready for the model. LightGBM accepts categorical values, but they have to be encoded as no-negative integers.


In [201]:
from sklearn.model_selection import train_test_split
from moscow_housing.display_data import import_data

#We dont want our model to care about the id of the house or the seller
#In my first run, i will replace missing values with the mean value
data, data_test = import_data()
Y = data.price
data = data.drop(columns=['price','id','seller'])


for column in data.columns:
    column_type = data[column].dtype
    if column_type == 'object':
        break
    data[column] = data[column].replace(np.NaN, data[column].mean())

#turn categorical features into correct type
for column in data.columns:
    column_type = data[column].dtype
    if column_type == 'object':
        data[column] = data[column].astype('category')


X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)


def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5


lightGBM_model = lgb.LGBMRegressor(
    num_leaves=20,
    max_depth=15,
    random_state=42,
    silent=True,
    metric='mse',
    n_jobs=4,
    n_estimators=10000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.09)

lightGBM_model.fit(X_train,y_train)
prediction = lightGBM_model.predict(X_test)

count = 0
for i in prediction:
    if i < 0:
        prediction[count] = prediction.mean()
    count += 1

rmsle = root_mean_squared_log_error(y_test,prediction)
print("first run", rmsle)



first run 0.1834119728836997


Second attempt

In [202]:
from sklearn.model_selection import train_test_split
from moscow_housing.display_data import import_data

#We dont want our model to care about the id of the house or the seller
#For my second run, i want to use linear regression to predict the missing values
data, data_test = import_data()
data.drop(columns=['id', 'seller' ])

#turn categorical features into correct type
for column in data.columns:
    column_type = data[column].dtype
    if column_type == 'object':
        print(column_type)
        data[column] = data[column].astype('category')

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


imp_mean = IterativeImputer(estimator=RandomForestRegressor, random_state=0)
imp_mean.fit(data)
transformed_data = imp_mean.transform(data)





data = transformed_data.drop(columns='price')

Y = data.price

X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)


def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5


lightGBM_model = lgb.LGBMRegressor(
    num_leaves=20,
    max_depth=15,
    random_state=42,
    silent=True,
    metric='mse',
    n_jobs=4,
    n_estimators=10000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.09)

lightGBM_model.fit(X_train,y_train)
prediction = lightGBM_model.predict(X_test)

count = 0
for i in prediction:
    if i < 0:
        prediction[count] = prediction.mean()
    count += 1

rmsle = root_mean_squared_log_error(y_test,prediction)
print("second run", rmsle)

object
object


TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.